<h1 align="center"> Aplicações em Processamento de Linguagem Natural </h1>
<h2 align="center"> Aula 09 - Geração Automática de Texto </h2>
<h3 align="center"> Prof. Fernando Vieira da Silva MSc.</h3>

<h2> 1. Introdução</h2>

<p>Enquanto na Extração de Informação a tarefa principal estava relacionada em compreender a escrita e tratar ambiguidades ou similaridades no texto, na geração automática de texto o desafio está em decidir como expressar uma dada informação em palavras.
    </p>
    
  <p>Há diversas aplicações práticas para NLG (do inglês, Natural Language Generation), como:</p>
  
  * Sumarização de documentos;
  * Geração de notícias;
  * Geração de texto para chatbots.
  
  <p>Nesta aula vamos abordar algumas técnicas para geração automática de texto, usando redes neurais recorrentes.</p>

<h2>2. Técnicas para Geração de Texto</h2>

<p> O objetivo da técnica de geração automática de texto é, dada uma sequência de n-gramas (ou caracteres, ou sentenças), prever qual será o próximo n-grama (ou caractere, ou sentença).
    </p>

<h2>3. Redes Neurais Recorrentes</h2>

<p>Uma rede neural recorrente é uma rede com um "loop", ou seja, uma rede que aproveita o aprendizado de uma amostra anterior para amostras seguintes. A figura abaixo, retirada de http://colah.github.io/posts/2015-08-Understanding-LSTMs/ ilustra sua arquitetura.
    </p>
    
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)    

<p>Perceba que essa arquitetura de redes neurais é especialmente aplicável para problemas com sequências, uma vez que as amostras anteriores influenciam na aprendizagem.</p>

<p>Porém, as redes recorrentes convencionais tem limitações para utilizar informações aprendidas em etapas mais distantes (que acabam sendo atualizadas durante o treino). As redes neurais LSTM vieram para tentar sanar esse problema, como veremos à seguir.</p>

<h3>Redes LSTM</h3>

<p>As redes LSTM (Long-Short Term Memory, ou de Memória de curto e longo prazo, numa tradução livre), ao contrário das redes neurais recorrentes convencionais, que apenas atualizam o estado aprendido em etapas anteriores, trazem o conceito de memória, que é controlada pelo modelo.
    </p>
    <p>Desta forma, esse modelo não apenas armazena os estados aprendidos em etapas anteriores, mas decide o que deve ser aproveitado, descartado ou atualizado. Vejamos a ilustração abaixo, também extraída de http://colah.github.io/posts/2015-08-Understanding-LSTMs/.
    </p>
    
![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)    

<p>As etapas ilustradas são: </p>
1. *Qual estado de memória deve ser lembrado*: A primeira camada usa uma função sigmoid que retorna um valor entre 0 e 1 para decidir se o estado de memória vindo da iteração anterior deve ser mantido (1 para completamente mantido) ou esquecido (0 para completamente esquecido);
2. *Qual estado de memória deve ser atualizado*: Primeiramente usa-se uma função sigmoid para decidir quais valores serão atualizados com os novos dados da amostra atual, depois usa-se uma função tanh (que retorna um valor entre -1 e 1) para criar um novo vetor de estados da memória(denominado Ct) que poderá ser utilizado na iteração seguinte;
3. *Atualizar as células de memória antigas*: Combina os estados que devem ser lembrados, os estados que devem ser atualizados e os novos estados para atualizar as células de memória, que serão utilizadas na iteração seguinte;
4. *Decide a saída*: Por fim, a saída da iteração é feita através da combinação de uma função sigmoid e uma função tanh, considerando o estado atual da memória.

    

<h2>4. Usando LSTM para geração de texto</h2>

<p> Vamos nos basear no exemplo disponível em https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms, e vamos usar um LSTM para geração de texto.
    </p>

In [1]:
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, Activation
import keras.utils as kutils

Using TensorFlow backend.


In [ ]:
#import nltk
#nltk.corpus.gutenberg.fileids()


In [2]:
#raw_txt = nltk.corpus.gutenberg.raw("shakespeare-hamlet.txt") + \
#nltk.corpus.gutenberg.raw("shakespeare-caesar.txt") + \
#nltk.corpus.gutenberg.raw("shakespeare-macbeth.txt")

# Código adaptado de https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms
import pandas as pd
import os

curr_dir = '../input/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

1214

In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string

all_sents = [[w.lower() for w in word_tokenize(sen) if not w in string.punctuation] \
             for sen in all_headlines]

x = []
y = []

print(all_sents[:10])

for sen in all_sents:
    for i in range(1, len(sen)):
        x.append(sen[:i])
        y.append(sen[i])
        

print(x[:10])
print(y[:10])



[['former', 'n.f.l', 'cheerleaders', '’', 'settlement', 'offer', '1', 'and', 'a', 'meeting', 'with', 'goodell'], ['e.p.a', 'to', 'unveil', 'a', 'new', 'rule', 'its', 'effect', 'less', 'science', 'in', 'policymaking'], ['the', 'new', 'noma', 'explained'], ['how', 'a', 'bag', 'of', 'texas', 'dirt', 'became', 'a', 'times', 'tradition'], ['is', 'school', 'a', 'place', 'for', 'self-expression'], ['commuter', 'reprogramming'], ['ford', 'changed', 'leaders', 'looking', 'for', 'a', 'lift', 'it', '’', 's', 'still', 'looking'], ['romney', 'failed', 'to', 'win', 'at', 'utah', 'convention', 'but', 'few', 'believe', 'he', '’', 's', 'doomed'], ['chain', 'reaction'], ['he', 'forced', 'the', 'vatican', 'to', 'investigate', 'sex', 'abuse', 'now', 'he', '’', 's', 'meeting', 'with', 'pope', 'francis']]
[['former'], ['former', 'n.f.l'], ['former', 'n.f.l', 'cheerleaders'], ['former', 'n.f.l', 'cheerleaders', '’'], ['former', 'n.f.l', 'cheerleaders', '’', 'settlement'], ['former', 'n.f.l', 'cheerleaders', 

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np

all_text = [c for sen in x for c in sen]
all_text += [c for c in y]

all_text.append('UNK') # Palavra desconhecida

words = list(set(all_text))
        
word_indexes = {word: index for index, word in enumerate(words)}      

max_features = len(word_indexes)

x = [[word_indexes[c] for c in sen] for sen in x]
y = [word_indexes[c] for c in y]

print(x[:10])
print(y[:10])

y = kutils.to_categorical(y, num_classes=max_features)

maxlen = max([len(sen) for sen in x])

print(maxlen)


[[2574], [2574, 863], [2574, 863, 1515], [2574, 863, 1515, 3185], [2574, 863, 1515, 3185, 1998], [2574, 863, 1515, 3185, 1998, 1523], [2574, 863, 1515, 3185, 1998, 1523, 1203], [2574, 863, 1515, 3185, 1998, 1523, 1203, 846], [2574, 863, 1515, 3185, 1998, 1523, 1203, 846, 1595], [2574, 863, 1515, 3185, 1998, 1523, 1203, 846, 1595, 764]]
[863, 1515, 3185, 1998, 1523, 1203, 846, 1595, 764, 3392]
23


In [5]:
x = pad_sequences(x, maxlen=maxlen)
x = pad_sequences(x, maxlen=maxlen)

print(x[:10,-10:])
print(y[:10,-10:])

[[   0    0    0    0    0    0    0    0    0 2574]
 [   0    0    0    0    0    0    0    0 2574  863]
 [   0    0    0    0    0    0    0 2574  863 1515]
 [   0    0    0    0    0    0 2574  863 1515 3185]
 [   0    0    0    0    0 2574  863 1515 3185 1998]
 [   0    0    0    0 2574  863 1515 3185 1998 1523]
 [   0    0    0 2574  863 1515 3185 1998 1523 1203]
 [   0    0 2574  863 1515 3185 1998 1523 1203  846]
 [   0 2574  863 1515 3185 1998 1523 1203  846 1595]
 [2574  863 1515 3185 1998 1523 1203  846 1595  764]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
print(x[:10,-10:])

for y_ in y:
    for i in range(len(y_)):
        if y_[i] != 0:
            print(i)

[[   0    0    0    0    0    0    0    0    0 2574]
 [   0    0    0    0    0    0    0    0 2574  863]
 [   0    0    0    0    0    0    0 2574  863 1515]
 [   0    0    0    0    0    0 2574  863 1515 3185]
 [   0    0    0    0    0 2574  863 1515 3185 1998]
 [   0    0    0    0 2574  863 1515 3185 1998 1523]
 [   0    0    0 2574  863 1515 3185 1998 1523 1203]
 [   0    0 2574  863 1515 3185 1998 1523 1203  846]
 [   0 2574  863 1515 3185 1998 1523 1203  846 1595]
 [2574  863 1515 3185 1998 1523 1203  846 1595  764]]
863
1515
3185
1998
1523
1203
846
1595
764
3392
367
807
1660
1595
1941
1990
2481
1693
1426
1262
3128
2919
1941
128
2759
1595
2012
2024
2277
1755
2158
1595
3296
2970
166
1595
201
2701
772
84
2391
1093
984
2701
1595
544
1621
3185
3407
555
984
1069
807
913
2511
3434
2815
2254
2568
2681
1884
3185
3407
430
3313
783
1480
173
807
518
1050
1808
109
1884
3185
3407
764
3392
1441
2781
3119
2583
1256
1595
224
3277
3034
344
971
2021
3213
1675
910
1318
296
691
1585
1830
1901
901


1537
1299
3438
1807
397
3346
1712
1480
1826
3185
2711
1116
2183
2225
2956
2842
237
3128
2136
2113
2042
1565
3321
49
846
3357
2064
2550
2024
2551
3246
1291
1610
1595
1881
2454
3185
2866
2392
3317
1480
2768
3185
1228
2270
1214
3349
2521
1222
1257
2017
3185
3407
1136
135
2929
3302
41
25
846
28
1519
2024
2257
10
2110
3276
3215
3291
1326
708
2317
928
747
1039
2024
2206
2560
3185
807
2195
2902
1247
2317
867
3185
3407
2178
2181
1992
2110
840
450
929
12
3148
3253
189
1467
3302
1480
2878
2024
1535
3185
3407
1095
3262
1595
2677
807
748
846
748
846
748
1670
2621
3029
2754
1984
1480
3196
2859
1299
2125
1692
1934
807
2633
2010
846
1951
2024
1022
944
1927
2444
807
2651
2511
353
2024
1299
1131
519
901
516
2966
3203
3014
1654
2133
3185
3407
1309
3128
660
550
433
70
446
1040
504
3153
3128
1941
1378
749
846
424
2937
1781
3185
3407
161
1480
2666
840
891
2773
296
1090
3128
1222
2504
3253
1153
930
1889
1222
546
929
1466
1260
759
2024
1703
2701
1094
3292
2433
3185
181
3013
807
539
327
2024
844
2307
26
1134


1378
3185
3407
683
3128
3138
559
1982
807
1235
36
3302
1595
1213
2329
3380
1595
2206
2323
3185
386
485
3185
296
1824
1340
970
3406
1781
1120
3253
296
1595
3003
3302
1480
2838
1941
1095
2990
2317
1480
1309
3064
2172
2024
2601
807
3176
3408
2739
2024
1480
746
2511
2244
1311
309
846
559
2421
161
2839
2295
3161
3302
1484
1270
1580
2332
3392
1216
2904
807
1057
1763
1665
2024
1595
1142
1607
1595
1732
164
296
183
1595
1732
164
1595
1167
1164
2254
2431
1480
2594
2802
3403
2317
1968
3419
3128
41
3059
1565
1220
302
36
1909
1487
3185
3407
85
2621
3283
3185
2866
447
1050
3418
2800
3130
480
45
2853
2244
3138
1439
925
1396
2821
1480
768
28
296
3137
1621
2752
1322
3253
3128
1595
1667
157
969
1480
2449
2024
2042
184
1837
1293
3425
1032
1621
3185
3407
3253
1840
234
2701
3304
3128
410
846
996
154
792
2701
1537
3392
1394
1514
1036
3392
658
731
1691
724
2915
1890
1971
86
3114
489
191
2181
3185
1565
1211
880
2561
2024
2803
1595
201
846
1595
3044
2772
807
1404
3165
1073
2850
2356
3014
414
1799
2392
1691
250

2819
992
2588
2564
926
1947
956
1547
1480
2574
1264
3032
3185
3407
3081
1924
846
1183
397
1668
875
1488
3128
898
1727
477
204
1895
3021
3120
1439
1884
723
755
2044
1329
1781
2292
3182
1595
2921
3327
87
748
2274
539
268
2332
1826
611
3334
2860
3185
2866
493
1171
2038
365
2701
2974
2028
3185
3407
2388
296
2377
3128
1733
353
2024
2868
911
2706
3185
3407
3191
328
3185
2711
3134
2183
2098
68
296
1808
1266
1489
1908
802
1593
1323
190
3185
3407
2355
1457
1896
1367
1166
1224
2009
3392
3317
2511
2373
276
3421
3432
2284
2114
811
2962
3242
161
1480
1559
781
1028
3302
464
191
2752
2009
2432
2745
754
36
3302
2335
2149
3185
3407
2243
1297
2701
2826
725
17
1595
1859
2024
1221
3128
2613
497
2835
3185
3407
673
1480
2957
807
2944
1694
1781
1991
1595
2405
577
3424
3142
3185
2866
3343
1751
691
119
3353
2973
807
1702
1439
459
1439
2404
137
1063
1696
3297
1696
807
1595
1941
2322
2024
1595
1175
396
2360
1474
1595
1393
1333
1595
298
1395
1686
182
1806
191
859
1832
2144
2009
3411
2256
2024
3253
3412
807
3194
1

359
353
1203
846
499
3233
1302
2701
2206
1376
3221
1439
1497
1439
297
2993
1684
3437
2464
1003
2855
3185
191
1780
807
1984
231
3297
1832
1738
3185
1772
2701
191
1192
1140
1107
3185
3407
2206
599
3128
1480
2035
3185
3185
1586
1595
3368
998
3079
2752
860
2390
1595
1083
3128
853
2237
1480
2125
3385
1066
2154
571
3185
3407
459
2600
191
1842
1595
1118
747
191
859
671
263
2392
2701
1480
1257
2042
1033
73
193
3123
2701
1595
2387
509
147
807
1480
321
2024
1556
590
3185
3407
855
2243
3111
846
1101
1393
1625
1673
807
922
385
952
1480
2796
807
1751
1464
3226
1045
611
800
261
2175
2191
2752
2848
3392
650
1701
191
2712
1894
1781
1896
2024
115
3128
2855
138
3185
2694
296
1597
1191
3128
1737
882
2024
2631
504
3153
3128
1941
1378
234
846
3236
2428
3128
2542
660
554
1231
1439
1621
1365
1480
2552
3185
3407
2178
3302
3165
3185
3407
2080
1979
2701
1480
455
2181
1480
2868
3185
3407
3382
590
929
2712
539
2748
2701
2206
855
1958
3000
3185
3010
807
2574
2360
2968
3115
1439
863
841
2564
1686
3185
3407
2477
202

<p>Agora vamos criar nosso modelo de LSTM usando o Keras.</p>

In [7]:
embedding_size = 10

model = Sequential()
    
# Add Input Embedding Layer
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    
# Add Hidden Layer 1 - LSTM Layer
model.add(LSTM(100))
model.add(Dropout(0.1))
    
# Add Output Layer
model.add(Dense(max_features, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 23, 10)            34420     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3442)              347642    
Total params: 426,462
Trainable params: 426,462
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(x, y, epochs=20, verbose=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [10]:
import pickle

print("Saving model...")
model.save('shak-nlg.h5')

with open('shak-nlg-dict.pkl', 'wb') as handle:
    pickle.dump(word_indexes, handle)

with open('shak-nlg-maxlen.pkl', 'wb') as handle:
    pickle.dump(maxlen, handle)
print("Model Saved!")

Saving model...
Model Saved!


In [11]:
import pickle

model = keras.models.load_model('shak-nlg.h5')
maxlen = pickle.load(open('shak-nlg-maxlen.pkl', 'rb'))
word_indexes = pickle.load(open('shak-nlg-dict.pkl', 'rb'))

<p>Agora vejamos como o algoritmo prevê uma palavra simples.</p>

In [ ]:
sample_seed = input()
sample_seed_vect = np.array([[word_indexes[c] if c in word_indexes.keys() else word_indexes['UNK'] \
                    for c in word_tokenize(sample_seed)]])

print(sample_seed_vect)

sample_seed_vect = pad_sequences(sample_seed_vect, maxlen=maxlen)

print(sample_seed_vect)

predicted = model.predict_classes(sample_seed_vect, verbose=0)

print(predicted)

def get_word_by_index(index, word_indexes):
    for w, i in word_indexes.items():
        if index == i:
            return w
        
    return None


for p in predicted:    
    print(get_word_by_index(p, word_indexes))

<p><b>Exercício 9</b>: Escreva um algoritmo que gera palavras até 100 palavras, com base em um texto curto de entrada (no máximo 5 palavras).</p>

In [ ]:
sample_seed = input()
sample_seed_vect = np.array([[word_indexes[c] if c in word_indexes.keys() else word_indexes['UNK'] \
                    for c in word_tokenize(sample_seed)]])

print(sample_seed_vect)

predicted=[]

sample_seed_vect = pad_sequences(sample_seed_vect, maxlen=maxlen)

print(sample_seed_vect)

while (len(sample_seed_vect)<100):
    predicted = model.predict_classes(pad_sequences([sample_seed_vect]), verbose=0)
    sample_seed_vect.extend(predicated)
    print(predicted)

res = []
for w in sample_seed_vect:
    res.append(get_word_by_index(w,word_indexs))
    
print(' '.join(res))